In [1]:
import pandas
import spotipy
import redis
from spotipy.oauth2 import SpotifyClientCredentials
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
# Set spotipy object
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id='118aa19f2b66476fbc062f0ac146d8b5',
    client_secret='7ca95a3159ab4391bee70f70d47a9271'
))

In [3]:
# Connetto a redis e lancio un ping
REDIS_HOST = "jupyter_redis"
REDIS_PORT = 6379
REDIS = redis.Redis(host=REDIS_HOST, port=REDIS_PORT)
# test the connection
REDIS.ping()

True

# Prove

In [9]:
sp.audio_features('1239722322')

[None]

In [7]:
print(REDIS.get('wdfga')==None)

True


### Connessione POSTGRES da spark

In [15]:
from pyspark.sql.functions import to_timestamp, date_format
import pyspark.sql.functions as f

spark = SparkSession \
    .builder \
    .appName('DBAnalysis') \
    .config('spark.driver.extraClassPath', 'postgresql-42.2.10.jar') \
    .getOrCreate()

In [16]:
properties = {
    'driver': 'org.postgresql.Driver',
    'url': 'jdbc:postgresql://postgres:5432/postgres',
    'user': 'postgres',
    'password': 'postgres1234',
    'dbtable': ' spotify_details',
}

In [17]:
df2 = spark.read \
    .format('jdbc') \
    .option('driver', properties['driver']) \
    .option('url', properties['url']) \
    .option('user', properties['user']) \
    .option('password', properties['password']) \
    .option('dbtable', properties['dbtable']) \
    .load()

In [18]:
df2.printSchema()

root
 |-- id_playlist: string (nullable = true)
 |-- name_playlist: string (nullable = true)
 |-- id_track: string (nullable = true)
 |-- name_track: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- time_signature: string (nullable = true)



In [19]:
# Creo la colonna year_month
df3 = df2.withColumn('year_month', date_format(
        to_timestamp(df2.timestamp, "yyyy-MM-dd'T'HH:mm:ssXXX"), 
        "yyyy-MM"
    ).alias('year_month')) 

In [20]:
# Aggrego sulla playlist e conto le ricorrenze
df4 = df3.groupby('id_playlist', 'year_month').count().sort('id_playlist', ascending=False)
df4.show(10)

+--------------------+----------+-----+
|         id_playlist|year_month|count|
+--------------------+----------+-----+
|7zvpNOLMErZqvGA4m...|   2020-12|   12|
|7zvpNOLMErZqvGA4m...|   2021-01|    1|
|7zvpNOLMErZqvGA4m...|   2020-10|   35|
|7zvpNOLMErZqvGA4m...|   2020-08|   22|
|7zvpNOLMErZqvGA4m...|   2020-11|   19|
|7zvpNOLMErZqvGA4m...|   2020-09|    2|
|7z3q0iLySqsrCJ0yu...|   2021-01|    2|
|7z3q0iLySqsrCJ0yu...|   2020-12|    3|
|7z3q0iLySqsrCJ0yu...|   2020-09|    3|
|7z3q0iLySqsrCJ0yu...|   2020-10|    4|
+--------------------+----------+-----+
only showing top 10 rows



In [21]:
# Per assegnare un mese alla playlist si é deciso di scegliere il mese con maggiori "aggiunte" di canzoni
# Per ogni playlist seleziono solo quella con ricorrenze per mese maggiore 
from pyspark.sql import Window

w = Window.partitionBy('id_playlist')

df5 = df4.withColumn('max', f.max('count').over(w))\
    .where(f.col('count') == f.col('max'))\
    .drop('max', 'count')

df5.sort('id_playlist', ascending=False)\
    .show()


+--------------------+----------+
|         id_playlist|year_month|
+--------------------+----------+
|7zvpNOLMErZqvGA4m...|   2020-10|
|7z3q0iLySqsrCJ0yu...|   2020-11|
|7z3hyKsoXNxKSYUCe...|   2020-12|
|7ymbfYgGdOqRns9yE...|   2020-04|
|7ygraCBBpUscXNrAc...|   2020-09|
|7yV3JkSqKv3GVTqmD...|   2020-04|
|7y87QYIUUmUGXJpMC...|   2020-12|
|7xiMMVwvFjqjMZ6qX...|   2020-11|
|7xO13RrgA0UIyysNf...|   2020-03|
|7xJkD9gWn8u0MnOqn...|   2019-12|
|7xFIxOjlXq2nFNGex...|   2019-01|
|7w1y3shCr8uDx1aKR...|   2020-03|
|7vhibs6g5WZL3PHKL...|   2020-06|
|7v9h1AMd65L8MCQaJ...|   2020-03|
|7ubkVtDuMGueTCxSy...|   2021-01|
|7uSDpfjUmrtjbv422...|   2020-07|
|7uDicALsURHtRq8lO...|   2020-05|
|7tqmR5oz5ApHFnS6f...|   2020-05|
|7tZeMUzIOBM6H1m7J...|   2020-11|
|7tZ8MM7p8F3cgyPPr...|   2020-05|
+--------------------+----------+
only showing top 20 rows



In [22]:
# Il df completo ha l'informazione sul mese di riferimento assegnata ad ogni playlist
df_complete = df2.join(df5, on=['id_playlist'], how='left')

df_complete\
.select('id_playlist', 'name_playlist','year_month')\
.show(10)

+--------------------+-------------+----------+
|         id_playlist|name_playlist|year_month|
+--------------------+-------------+----------+
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
|0AwlxqiHGBBs67GD4...|   F#*K COVID|   2020-12|
+--------------------+-------------+----------+
only showing top 10 rows



In [23]:
df_complete2 =  df_complete.groupBy("id_playlist", 'name_playlist' , 'year_month')\
.agg(f.mean('danceability'),f.stddev_pop('danceability'),f.mean('energy'),f.stddev_pop('energy'),f.mean('valence'),f.stddev_pop('valence'))\
.sort('year_month', ascending=True)

In [24]:
df_complete2.select('avg(danceability)','stddev_pop(danceability)').show(1000)

+-------------------+------------------------+
|  avg(danceability)|stddev_pop(danceability)|
+-------------------+------------------------+
| 0.7523764019739783|     0.10029577881212025|
| 0.6509364333652937|       0.142381056095038|
| 0.7312476190476187|     0.10270714547891151|
|0.48336964980544794|      0.1729158933139975|
| 0.6928454208575969|      0.1361570280810571|
| 0.7658982558139534|     0.10765442449447626|
| 0.6440680786686838|     0.13315948791988527|
| 0.5216804123711342|     0.15540457429736335|
| 0.6845075376884431|     0.10029804175238993|
| 0.5525405405405405|     0.15215353695170886|
| 0.4438835341365461|     0.10682804601612038|
| 0.5859059602649006|      0.1244849113076891|
| 0.7593415637860079|     0.13087305208546038|
|  0.586126315789474|     0.12702927487748744|
| 0.7321872483221483|     0.09598771159368279|
|  0.620443298969072|     0.13348994858273777|
| 0.7550560224089636|      0.0830796061083919|
|  0.648693825042881|     0.12755909973080737|
| 0.636631782

In [25]:
df_complete2.select('avg(valence)','stddev_pop(valence)').show(1000)

+-------------------+-------------------+
|       avg(valence)|stddev_pop(valence)|
+-------------------+-------------------+
|  0.486153978906999|0.23893464502551406|
| 0.7105960969044416|0.16834721553395637|
| 0.8984500000000004|0.11377134493611045|
| 0.5115595330739297| 0.2595165712165001|
| 0.5215991529909993|0.22728163432525458|
| 0.6196502906976739|0.17537279909582962|
| 0.7046030257186072| 0.1887689098399449|
| 0.5328670103092779|0.26047999971569114|
| 0.5482718592964828| 0.1957200408337894|
| 0.6057144144144145|0.20608776309749924|
|0.19954979919678725|0.12620733121244146|
| 0.2723017218543046|0.17914105074845466|
| 0.5451876543209878|0.21438383237597403|
|0.47321000000000013|0.22161434747160025|
| 0.7097522147650998|0.19593323842042115|
| 0.5074216494845362|0.21505597843630556|
| 0.6354621848739491| 0.1784984581177338|
|  0.514698799313894|0.22140816153684997|
|  0.743748062015504|0.19924126063168912|
| 0.6642810298102984|0.18190726774231994|
|0.46214112149532705| 0.2255386267

In [26]:
df_complete2.select('avg(energy)','stddev_pop(energy)').show(1000)
# Dai risultati vediamo come le playlist non sono poi cosí variabili
# Probabilmente per il concetto stesso di playlist

+-------------------+-------------------+
|        avg(energy)| stddev_pop(energy)|
+-------------------+-------------------+
| 0.5944983849259768|0.14772635444699458|
| 0.7278868648130394|0.14812054460306004|
| 0.8444642857142856| 0.1259911794369684|
|0.46534852140077837|0.23220261583566745|
| 0.6586084700899962|0.16726858132286893|
| 0.7338982558139536|0.11002747329952982|
| 0.5906691376701966|0.19020124914047426|
| 0.4138025429553265|0.22626164805632054|
| 0.8025326633165832|0.10240935537726102|
| 0.6583513513513514|0.24250147760852506|
| 0.9134979919678715|0.07271057907361775|
| 0.6631522633744856|0.13726336714468906|
| 0.8932105960264901|0.09118360427757134|
| 0.6109363157894737|0.20099070991127213|
| 0.7521758389261752|0.12632940304600762|
| 0.6422783505154641|0.17710901426342385|
| 0.7259887955182068|0.10506212588096464|
| 0.7531493996569478| 0.1411220388817074|
| 0.6284108527131783|0.17066541510577468|
| 0.7371924119241192|0.09916299219472893|
|  0.635570093457944|0.14522924914

In [28]:
df_complete2.filter(df_complete2.year_month < '2020-01').count()

89